In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

In [2]:
data=pd.read_csv("accident_train.csv")

In [3]:
data.columns

Index(['Unnamed: 0', 'Severity', 'Start_Lat', 'Start_Lng', 'Temperature(F)',
       'Wind_Chill(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)',
       'Wind_Direction', 'Wind_Speed(mph)', 'Precipitation(in)',
       'Weather_Condition', 'Amenity', 'Bump', 'Crossing', 'Junction',
       'Station', 'Stop', 'Traffic_Signal', 'Sunrise_Sunset'],
      dtype='object')

In [4]:
if data.columns[0].startswith('Unnamed'):
    data = data.drop(data.columns[0], axis=1)

In [16]:
data = pd.get_dummies(data, columns=['Weather_Condition', 'Wind_Direction'], drop_first=True)

# Split dataset into features (X) and target (y)
X = data.drop(columns=['Severity'])
y = data['Severity']

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X.iloc[1:,2:9])
X.iloc[1:, 2:9] = imputer.transform(X.iloc[1:, 2:9])
# Convert target to one-hot encoding
y_one_hot = to_categorical(y)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.01, random_state=42)


In [18]:
X_train

,Start_Lat,Start_Lng,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Speed(mph),Precipitation(in),Amenity,...,Wind_Direction_SSE,Wind_Direction_SSW,Wind_Direction_SW,Wind_Direction_South,Wind_Direction_VAR,Wind_Direction_Variable,Wind_Direction_W,Wind_Direction_WNW,Wind_Direction_WSW,Wind_Direction_West
2362,37.250462,-121.911987,59.0,59.000000,83.0,29.90,10.0,8.000000,0.000000,False,...,False,False,False,False,False,False,False,False,False,False
51010,33.084515,-80.652550,84.2,58.040291,66.0,29.93,10.0,4.600000,0.008407,False,...,False,False,False,False,False,False,False,False,False,False
18053,41.200981,-80.071564,30.9,26.200000,79.0,30.35,10.0,4.600000,0.008407,False,...,False,False,False,False,False,True,False,False,False,False
72164,41.184433,-79.541534,57.0,57.000000,100.0,28.41,0.0,0.000000,0.000000,False,...,False,False,False,False,False,False,False,False,False,False
49024,39.630966,-104.902405,59.0,58.040291,22.0,29.93,10.0,7.690208,0.008407,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,45.129771,-93.188739,-11.0,-26.000000,64.0,29.57,10.0,7.000000,0.000000,False,...,False,False,False,False,False,False,False,True,False,False
54886,47.663250,-122.299911,45.0,58.040291,74.0,30.28,10.0,7.690208,0.008407,True,...,False,False,False,False,False,False,False,False,False,False
76820,40.074249,-75.309669,33.8,30.800000,75.0,30.16,10.0,3.500000,0.008407,False,...,False,False,False,False,False,False,False,True,False,False
860,35.079208,-90.057091,86.0,86.000000,63.0,29.76,10.0,0.000000,0.000000,False,...,False,False,False,False,False,False,False,False,False,False


In [20]:
y_train

array([[0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])

In [24]:
# Build Neural Network Model
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),    
    Dense(32, activation='relu'),
    Dropout(0.2),    
    Dense(5, activation='softmax')  # 3 output classes for 'Low', 'Medium', 'High'
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


C:\Users\Supratim\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/10
2475/2475 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.7822 - loss: 1.0740 - val_accuracy: 0.7964 - val_loss: 0.6236
Epoch 2/10
2475/2475 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - accuracy: 0.7963 - loss: 0.6315 - val_accuracy: 0.7964 - val_loss: 0.6232
Epoch 3/10
2475/2475 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.7963 - loss: 0.6257 - val_accuracy: 0.7964 - val_loss: 0.6234
Epoch 4/10
2475/2475 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - accuracy: 0.7970 - loss: 0.6230 - val_accuracy: 0.7964 - val_loss: 0.6232
Epoch 5/10
2475/2475 ━━━━━━━━━━━━━━━━━━━━ 20s 6ms/step - accuracy: 0.7977 - loss: 0.6189 - val_accuracy: 0.7964 - val_loss: 0.6232
Epoch 6/10
2475/2475 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - accuracy: 0.7949 - loss: 0.6221 - val_accuracy: 0.7964 - val_loss: 0.6228
Epoch 7/10
2475/2475 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.7996 - loss: 0.6124 - val_accuracy: 0.7964 - val_loss: 0.6228
Epoch 8/10
2475/2475 ━━━━━━━━━━━━━━━━━━━━ 24s 6ms/step - accuracy: 0.7925 - loss: 0

In [34]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.2f}")


Test Accuracy: 0.79


In [38]:
# Predict and evaluate
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

print("=== Neural Network Classification Report ===")
print(classification_report(y_test_classes, y_pred_classes))
print(f"Overall Accuracy: {accuracy_score(y_test_classes, y_pred_classes):.2f}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
=== Neural Network Classification Report ===
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         6
           2       0.79      1.00      0.88       788
           3       0.00      0.00      0.00       176
           4       0.00      0.00      0.00        30

    accuracy                           0.79      1000
   macro avg       0.20      0.25      0.22      1000
weighted avg       0.62      0.79      0.69      1000

Overall Accuracy: 0.79


C:\Users\Supratim\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Supratim\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Supratim\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [72]:
model.predict(X_test)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[1.3533410e-08, 7.7278032e-03, 8.0513847e-01, 1.6085438e-01,
        2.6279263e-02],
       [1.3533410e-08, 7.7278032e-03, 8.0513847e-01, 1.6085438e-01,
        2.6279263e-02],
       [1.3533410e-08, 7.7278032e-03, 8.0513847e-01, 1.6085438e-01,
        2.6279263e-02],
       ...,
       [1.3533410e-08, 7.7278032e-03, 8.0513847e-01, 1.6085438e-01,
        2.6279263e-02],
       [1.3533410e-08, 7.7278032e-03, 8.0513847e-01, 1.6085438e-01,
        2.6279263e-02],
       [1.3533410e-08, 7.7278032e-03, 8.0513847e-01, 1.6085438e-01,
        2.6279263e-02]], dtype=float32)

In [80]:
# Example custom input values
custom_values = [[
    45.488880, -122.594900, 68.0, 68.0, 39.0, 30.03, 10, 8.1, 0.00840747,
    1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0
]]

# Step 1: Load the scaler used during training (if applicable)
scaler = StandardScaler()
scaler.fit(X_train)  # X_train should be the training dataset used to fit the scaler

# Step 2: Scale the custom input
scaled_input = scaler.transform(custom_values)

# Step 3: Predict using the neural network
predictions = model.predict(scaled_input)

# Step 4: Interpret the predictions
# For classification: Argmax for class labels
predicted_class = np.argmax(predictions, axis=1)

# For regression: Direct output
print("Predictions:", predictions)
print("Predicted Class:", predicted_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step

C:\Users\Supratim\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Predictions: [[1.3533411e-08 7.7277999e-03 8.0513853e-01 1.6085437e-01 2.6279265e-02]]
Predicted Class: [2]


In [84]:
model.save('traffic_model.h5')